## Классификация поломок

Устанавливаем три библиотеки Python:

catboost: Для построения модели машинного обучения.

featuretools: Для автоматического создания новых признаков из данных.

optuna: Для поиска наилучших настроек для модели.

Эти библиотеки используются вместе для создания модели для предсказания поломок автомобилей на основе предоставленных данных.

In [1]:
!pip install catboost -q
!pip install featuretools -q
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.5 MB/s eta 0:00:00


Отключаем вывод предупреждений о потенциальных проблемах, чтобы не загромождать вывод программы.

In [2]:
import warnings

warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)

In [3]:
import featuretools as ft
import numpy as np
import optuna
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from woodwork.logical_types import Age, Categorical, Datetime

In [4]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

Этот код подготавливает данные и параметры для создания модели, которая будет предсказывать "target_class" автомобиля на основе других его характеристик.

In [5]:
N_SPLITS = 3
N_TRIALS = 50
RANDOM_SEED = 42
SEARCH_BEST_PARAMS = False

CAT_FEATURES = ["model", "car_type", "fuel_type"]
TARGET_COL = "target_class"

FEATURES_TO_DROP = ["car_id", "target_reg", "deviation_normal_count"]

## Загрузка данных

Загружаем данные о машинах из 5 файлов. Затем разделяет данные на обучающую и тестовую выборки. Удаляет столбец с ответом (target_class) из обучающей выборки и объединяет обучающую и тестовую выборки для дальнейшей обработки.

В итоге, код подготавливает данные для создания модели, которая будет предсказывать поломки автомобилей.

In [6]:
car_info_train = pd.read_csv("car_train.csv")
car_info_test = pd.read_csv("car_test.csv")
rides_info = pd.read_csv("rides_info.csv")
driver_info = pd.read_csv("driver_info.csv")
fix_info = pd.read_csv("fix_info.csv")

train_cars = car_info_train["car_id"]
test_cars = car_info_test["car_id"]
y_train = car_info_train[TARGET_COL]

car_info_train = car_info_train.drop(columns=[TARGET_COL, "target_class"])
all_data = pd.concat([car_info_train, car_info_test], ignore_index=True)

## Feature Engineering

Создаем новые признаки (фич) из данных, чтобы улучшить работу модели машинного обучения, которая будет предсказывать поломки автомобилей.

Что делает код:

1. Объединяет данные: Создает связи между разными таблицами (информация о машинах, поездках, водителях, ремонтах) на основе общих столбцов (например, car_id).
2. Создает новые признаки: Автоматически генерирует новые признаки, комбинируя информацию из связанных таблиц. Например, может создать признак "среднее количество поездок в месяц для данной модели машины".
3. Очищает признаки: Удаляет ненужные признаки, которые не несут полезной информации (например, признаки, имеющие только одно значение).
4. Готовит данные для модели: Разделяет данные на обучающую и тестовую выборки, удаляет ненужные столбцы и обновляет список категориальных признаков.

В результате: код создает новые, потенциально полезные признаки, которые помогут модели машинного обучения лучше предсказывать поломки автомобилей.

Ключевые моменты:

EntitySet: Контейнер, который хранит связанные таблицы.

ft.dfs: Функция, которая автоматически генерирует новые признаки.

CAT_FEATURES: Список категориальных признаков.

X_train, X_test: Матрицы признаков для обучения и тестирования модели.

In [7]:
# Создаём отношения между источниками данных
es = ft.EntitySet(id="car_data")

es = es.add_dataframe(
    dataframe_name="cars",
    dataframe=all_data,
    index="car_id",
    logical_types={
        "car_type": Categorical,
        "fuel_type": Categorical,
        "model": Categorical,
    },
)

es = es.add_dataframe(
    dataframe_name="rides",
    dataframe=rides_info.drop(["ride_id"], axis=1),
    index="index",
    time_index="ride_date",
)

es = es.add_dataframe(
    dataframe_name="drivers",
    dataframe=driver_info,
    index="user_id",
    logical_types={
        "sex": Categorical,
        "first_ride_date": Datetime,
        "age": Age,
    },
)

es = es.add_dataframe(
    dataframe_name="fixes",
    dataframe=fix_info,
    index="index",
    logical_types={
        "work_type": Categorical,
        "worker_id": Categorical,
    },
)

es = es.add_relationship("cars", "car_id", "rides", "car_id")
es = es.add_relationship("drivers", "user_id", "rides", "user_id")
es = es.add_relationship("cars", "car_id", "fixes", "car_id")

# Генерируем фичи
all_data, _ = ft.dfs(
    entityset=es,
    target_dataframe_name="cars",
    max_depth=2,
)

# Удаляем константные фичи
all_data = ft.selection.remove_single_value_features(all_data)

train_data = all_data.loc[train_cars].reset_index()
test_data = all_data.loc[test_cars].reset_index()

X_train = train_data.drop(columns=FEATURES_TO_DROP, errors="ignore")
X_test = test_data.drop(columns=FEATURES_TO_DROP, errors="ignore")

# Обновляем список категориальных фичей
CAT_FEATURES += [col for col in X_train if col.startswith("MODE")]

In [8]:
X_train.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,MAX(rides.deviation_normal),MAX(rides.distance),MAX(rides.rating),...,SKEW(rides.drivers.user_rides),SKEW(rides.drivers.user_time_accident),STD(rides.drivers.age),STD(rides.drivers.user_rating),STD(rides.drivers.user_rides),STD(rides.drivers.user_time_accident),SUM(rides.drivers.age),SUM(rides.drivers.user_rating),SUM(rides.drivers.user_rides),SUM(rides.drivers.user_time_accident)
0,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,0.001,1.849349e+06,9.44,...,0.751130,1.813293,10.109652,0.611473,546.505545,17.294174,5831.0,1432.0,144078.0,2056.0
1,VW Polo VI,economy,petrol,3.90,2015,78218,2021,47.673,2.762119e+06,10.00,...,0.555751,0.749350,9.592259,0.574089,585.134416,4.911820,6088.0,1390.0,160916.0,1212.0
2,Renault Sandero,standart,petrol,6.30,2012,23340,2017,4.001,1.744243e+06,9.70,...,0.374761,0.424371,9.277823,0.665963,531.541486,5.426470,5714.0,1364.7,163567.0,1701.0
3,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,48.956,2.167931e+06,10.00,...,0.780447,1.680088,8.458641,0.431415,514.264373,20.782287,6086.0,1483.2,165496.0,2299.0
4,Renault Sandero,standart,petrol,4.70,2012,26428,2017,49.269,2.167675e+06,9.94,...,0.448681,1.840095,10.327151,0.668640,570.284478,15.192322,5968.0,1411.5,155944.0,1891.0


In [9]:
X_test.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,MAX(rides.deviation_normal),MAX(rides.distance),MAX(rides.rating),...,SKEW(rides.drivers.user_rides),SKEW(rides.drivers.user_time_accident),STD(rides.drivers.age),STD(rides.drivers.user_rating),STD(rides.drivers.user_rides),STD(rides.drivers.user_time_accident),SUM(rides.drivers.age),SUM(rides.drivers.user_rating),SUM(rides.drivers.user_rides),SUM(rides.drivers.user_time_accident)
0,Skoda Rapid,economy,petrol,4.80,2013,42269,2019,36.661,3.021921e+06,8.91,...,0.735757,0.711919,10.280475,0.632886,580.542915,4.722760,5865.0,1373.6,147763.0,1665.0
1,Renault Sandero,standart,petrol,4.32,2015,90014,2016,0.085,2.279221e+06,10.00,...,1.238131,0.932452,10.115551,0.617048,600.837062,4.874220,6064.0,1391.7,149210.0,1200.0
2,Smart ForTwo,economy,petrol,4.46,2015,82684,2017,3.567,1.828386e+06,10.00,...,0.670552,0.774293,10.166570,0.594423,596.865263,5.154786,6011.0,1396.8,163074.0,1315.0
3,Smart ForFour,economy,petrol,2.80,2014,68833,2021,0.002,1.262004e+06,9.68,...,0.988304,0.546549,9.414867,0.612586,528.869754,4.728072,5991.0,1364.3,152614.0,1550.0
4,Skoda Rapid,economy,petrol,6.56,2013,42442,2021,7.628,1.950412e+06,10.00,...,0.740067,0.800319,9.535007,0.574252,574.605576,5.318292,6106.0,1392.3,148542.0,1230.0


## Подбор гиперпараметров

In [10]:
def fit_catboost(trial, train, val):

    X_train, y_train = train
    X_val, y_val = val

    params = {
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 50),
        "border_count": trial.suggest_int("border_count", 32, 512),
        "random_strength": trial.suggest_float("random_strength", 1e-3, 10),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 10),
        "rsm": trial.suggest_float("rsm", 0.1, 1.0),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "leaf_estimation_method": trial.suggest_categorical("leaf_estimation_method", ["Newton", "Gradient"]),
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0.0, 20.0)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1)


    model = CatBoostClassifier(
        **params,
        verbose=0,
        thread_count=-1,
        random_seed=RANDOM_SEED,
        cat_features=CAT_FEATURES,
        eval_metric="AUC",
    )

    model.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        verbose=0,
    )

    preds = model.predict_proba(X_val)

    return model, preds



In [11]:
def objective(trial, X_train, y_train, return_models=False, **kwargs):
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)

    scores, models = [], []

    for train_idx, val_idx in kf.split(X_train):
        train = X_train.iloc[train_idx, :], y_train.iloc[train_idx]
        val = X_train.iloc[val_idx, :], y_train.iloc[val_idx]

        model, y_pred = fit_catboost(trial, train, val, **kwargs)
        scores.append(roc_auc_score(val[1], y_pred, multi_class="ovo"))
        models.append(model)

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [12]:
# Подготовка данных для CatBoost
X_train_catboost = X_train.copy()
X_test_catboost = X_test.copy()
for cat_feature in CAT_FEATURES:
    X_train_catboost[cat_feature] = X_train_catboost[cat_feature].astype(str)
    X_test_catboost[cat_feature] = X_test_catboost[cat_feature].astype(str)

if SEARCH_BEST_PARAMS:
    # Подбор гиперпараметров для CatBoost
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study.optimize(
        lambda trial: objective(trial, X_train_catboost, y_train),
        n_trials=N_TRIALS,
        show_progress_bar=True,
    )

In [13]:
# Вывод лучших параметров и метрик
if SEARCH_BEST_PARAMS:
    catboost_best_params = study.best_params
else:
    catboost_best_params = {
        "depth": 6,
        "learning_rate": 0.24463003068774805,
        "l2_leaf_reg": 22.762810145080902,
        "border_count": 278,
        "random_strength": 5.9799693384364065,
        "one_hot_max_size": 5,
        "rsm": 0.9812168569778881,
        "boosting_type": "Plain",
        "bootstrap_type": "MVS",
        "leaf_estimation_method": "Gradient",
    }


catboost_best_value, catboost_best_models = objective(
    optuna.trial.FixedTrial(catboost_best_params),
    X_train_catboost,
    y_train,
    return_models=True,
)

print(f"Best CatBoost ROC-AUC: {catboost_best_value}\n\n")
print("Best CatBoost params:")
print(*[f"{key}: {value}" for key, value in catboost_best_params.items()], sep="\n")

Best CatBoost ROC-AUC: 0.9999988587995657


Best CatBoost params:
depth: 6
learning_rate: 0.24463003068774805
l2_leaf_reg: 22.762810145080902
border_count: 278
random_strength: 5.9799693384364065
one_hot_max_size: 5
rsm: 0.9812168569778881
boosting_type: Plain
bootstrap_type: MVS
leaf_estimation_method: Gradient


## Финальное предсказание

In [14]:
# Получим предсказания на тестовой выборке
preds = [
    catboost_model.predict_proba(X_test_catboost)
    for catboost_model in catboost_best_models
]

mean_preds = np.mean(preds, axis=0)
y_pred = np.argmax(mean_preds, axis=1)

class_mapping = dict(
    zip(
        range(y_train.nunique()),
        sorted(y_train.unique()),
    )
)

test_data[TARGET_COL] = np.vectorize(class_mapping.get)(y_pred)
test_data[["car_id", TARGET_COL]].to_csv(f"submission.csv", index=False)

In [15]:
# Сохранение каждой модели из KFold
for i, model in enumerate(catboost_best_models):
    model_path = f"catboost_model_fold_{i}.cbm"
    model.save_model(model_path)
    print(f"Модель для фолда {i} сохранена в {model_path}")


Модель для фолда 0 сохранена в catboost_model_fold_0.cbm
Модель для фолда 1 сохранена в catboost_model_fold_1.cbm
Модель для фолда 2 сохранена в catboost_model_fold_2.cbm


In [ ]:
from catboost import CatBoostClassifier

# Загрузка моделей
loaded_models = []
for i in range(len(catboost_best_models)):  # Количество фолдов
    model_path = f"catboost_model_fold_{i}.cbm"
    model = CatBoostClassifier()
    model.load_model(model_path)
    loaded_models.append(model)

# Получение предсказаний
preds = [model.predict_proba(X_test_catboost) for model in loaded_models]
mean_preds = np.mean(preds, axis=0)
y_pred = np.argmax(mean_preds, axis=1)


In [ ]:
test_data[TARGET_COL] = np.vectorize(class_mapping.get)(y_pred)
test_data[["car_id", TARGET_COL]].to_csv("submission.csv", index=False)
print("Файл с предсказаниями сохранён: submission.csv")


Файл с предсказаниями сохранён: submission.csv
